In [57]:
# вариант очистки данных максимально приближенный к оригиналу 
import re 
import os

# очистка данных

important_symbols = ['№', '-', ')', '(', '.', ',', ':',';', '«', '»', ' ']
latin_letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz'

def cleanser(text):
 
    words = text.split()
    final = []
    final.append('\n')
    temp = ''

    for word in words: 
        for w in word:
            if w not in latin_letters:
                if w in important_symbols:
                    temp += w
                elif w.isalnum():
                    temp += w   
        final.append(temp)
        temp = ''
                        
    final = " ".join(final) # склеить список из строк
    final = re.sub(r' {1,}', ' ', final) # удалить много пробелов
    
    return (final)    


def clean_docs(docs):
    
    final_text = []
    for i, doc in enumerate(docs):
        # if i == 0 or i == 1:    # не трогать название документа
        #     final_text.append(doc)
        # else:
            clean_doc = cleanser(doc)
            final_text.append(clean_doc)

    return (final_text)

In [48]:
file = open('/Users/gleb/Desktop/InternetLab/internetlab/docs/row_docs/row_region/Стратегия_Алтайский_край_2023.txt', 'r', encoding='utf-8')

cleaned_docs = clean_docs(file)

# запись в файл
with open('/Users/gleb/Desktop/InternetLab/internetlab/docs/clean_docs/reg_clean/wdqdqw', "w", encoding="utf-8") as d:
    for i in cleaned_docs:
        d.write(i)

In [58]:
r_path = '/Users/gleb/Desktop/InternetLab/internetlab/docs/row_docs/row_region/'
c_path = '/Users/gleb/Desktop/InternetLab/internetlab/docs/clean_docs/reg_clean/'

raw_doc_list = os.listdir(r_path)
clean_doc_list = os.listdir(c_path)
for doc in raw_doc_list:
    if '.txt' in doc:
        file  = open(os.path.join(r_path, doc), 'r', encoding='utf-8')
        cleaned_docs = clean_docs(file)
        # запись в файл
    
        document = os.path.join(c_path, 'c_' + doc)
        with open(document, "w", encoding="utf-8") as d:
            for i in cleaned_docs:
                d.write(i)
        